In [21]:
import numpy as np
import os
from shutil import rmtree

In [22]:
# ~~~~~~~~~~~~~~~~~~ SET UP BASICS
header = """# total energy = -155.84630218 eV

CRYSTAL
PRIMVEC
8.04 0.0 0.0
0.0 8.527707781 0.0
0.0 0.0 31.370277041
PRIMCOORD
56 1
"""

In [23]:

translation_tensor = np.zeros((56, 6))
translation_tensor[-5:, 2] += -8.

hzeros = np.zeros((56, 3))

nsteps, dz = 21, .0
# nsteps, dz = 2, .0


# load source
source_folders = [f.name for f in os.scandir('source') if f.is_dir()]

# make destination folder
dest = 'vdw_ref_dft_xsfs'

In [24]:
translation_tensor[-5:, 2] #+= -6.

array([-8., -8., -8., -8., -8.])

In [25]:
translation_tensor.shape

(56, 6)

In [26]:
if os.path.exists(dest):
    rmtree(dest)
os.makedirs(dest)

for sf in source_folders:
    base_tensor = np.loadtxt(os.path.join('source', sf, 'POSCAR'),
                             skiprows=7, usecols=[0, 1, 2])
    base_tensor = np.hstack((base_tensor, hzeros))
    new_folder = os.path.join(dest, sf)
    os.makedirs(new_folder)
    for i in range(nsteps):
        tensor = i/(nsteps-1)*translation_tensor + base_tensor
        with open(os.path.join(new_folder, "{:03d}.xsf".format(i)), 'w') as f:
            f.write(header)
            for j in range(51):
                s = ['Pt'] + ["{:13.10f}".format(n)
                              for n in tensor[j]] + ['\n']
                f.write('\t'.join(s))
            for j in range(51, 55):
                s = ['H'] + ["{:13.10f}".format(n) for n in tensor[j]] + ['\n']
                f.write('\t'.join(s))
            s = ['C'] + ["{:13.10f}".format(n) for n in tensor[-1]] + ['\n']
            f.write('\t'.join(s))